# Ribonanza - Attempt 2

A second approach to the [Stanford Ribonanza problem](https://www.kaggle.com/competitions/stanford-ribonanza-rna-folding/) that builds off the first approach.

Major differences:
- use of different model architecture
- use of only filtered data (data in which SN_filter == 1)

Currently, this scores a 0.24796 MAE

![Score](image.png)

## Todo

- [ ] Figure out why validation MAE differs so much from submission MAE
- [x] Train more
- [ ] Figure out what the reactivity_error means
- [ ] Try a transformer, LSTM, or RNN architecture and take advantage of masking

## Setup

### Filesystem Setup

Your project directory should look like this:

- `(project directory)`
    - `ribonanza2.ipynb`
    - `train_data.csv`
    - `test_data.csv` (optional)

`train_data.csv` is the only file necessary for training, and it can be downloaded from the kaggle competition linked in the description.

`test_data.csv` is only necessary if you intend to make and submit predictions.

### Code Setup

In [1]:
# imports
import tensorflow as tf
import keras
import keras.layers as layers
import keras.optimizers as optimizers
import pandas
import numpy as np
from tqdm import tqdm
import seaborn
import os

2023-10-06 11:55:40.364087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 11:55:41.009177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# constants

# according to kaggle, this is the maximum # of reactivites to be used
NUM_REACTIVITIES = 457

# there are 4 different bases (AUCG)
NUM_BASES = 4

## Data Preprocessing

### Filter Data

In [3]:
def filter_data(out: str, key: str, value: str, file_name: str, force: bool):
    """
    Filters a file to only take datapoints
    whose values of `key` are `value`.

    Parameters:
        - out: str - the name of the file that will store the filtered datapoints
        - key: str - the name of the key to look at
        - value: str - the value that the key should have
        - file_name: str - the name of the file that contains all the datapoints.
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    if os.path.exists(out) and not force:
        print("File already exists, not doing any work")
        return

    count = 0

    # count how many lines we have in total
    with open(file_name) as file:
        line = file.readline()  # ignore the header
        line = (
            file.readline()
        )  # take the first line since we increment count in the loop
        while line != "":
            count += 1
            line = file.readline()

    # use that knowledge for a progress bar
    with open(file_name, "r") as file, open(out, "w") as outfile:
        # write the header
        header = file.readline()
        outfile.write(header)

        # get what index the SN_filter is
        SN_idx = header.split(",").index(key)

        # only take the approved filtered lines
        for _ in tqdm(range(count)):
            line = file.readline()
            temp = line.split(",")
            if temp[SN_idx] == value:
                outfile.write(line)


def filter_train_data(force: bool = False):
    """
    Filters the immense train_data.csv to only take datapoints
    whose SN_filter (Signal to Noise filter) is 1. In other words,
    we only take good reads. These filtered datapoints are then
    written to the file provided

    Parameters:
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    filter_data("train_data_filtered.csv", "SN_filter", "1", "train_data.csv", force)


def filter_2A3(force: bool = False):
    """
    Only take the 2A3 points

    Parameters:
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    filter_data(
        "train_data_2a3.csv",
        "experiment_type",
        "2A3_MaP",
        "train_data_filtered.csv",
        force,
    )


def filter_DMS(force: bool = False):
    """
    Only take the DMS points

    Parameters:
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    filter_data(
        "train_data_dms.csv",
        "experiment_type",
        "DMS_MaP",
        "train_data_filtered.csv",
        force,
    )

In [4]:
# filter our data
filter_train_data()

File already exists, not doing any work


In [5]:
# take the 2a3 points
filter_2A3()

File already exists, not doing any work


In [6]:
# take the dms points
filter_DMS()

File already exists, not doing any work


### Convert Data to Inputs and Outputs

In [7]:
# encode inputs as
# A : [1, 0, 0, 0]
# U : [0, 1, 0, 0]
# C : [0, 0, 1, 0]
# G : [0, 0, 0, 1]
base_map = {
    "A": np.array([1, 0, 0, 0]),
    "U": np.array([0, 1, 0, 0]),
    "C": np.array([0, 0, 1, 0]),
    "G": np.array([0, 0, 0, 1]),
}

In [8]:
# encode inputs as
# A : [1, 0, 0, 0]
# U : [0, 1, 0, 0]
# C : [0, 0, 1, 0]
# G : [0, 0, 0, 1]
base_map = {
    "A": np.array([1, 0, 0, 0]),
    "U": np.array([0, 1, 0, 0]),
    "C": np.array([0, 0, 1, 0]),
    "G": np.array([0, 0, 0, 1]),
}


def preprocess_csv(out: str, file_name: str, force: bool = False):
    """
    Preprocess the csv and save the preprocessed data as a .npz file

    Parameters:
        - out: str - the name of the file to save the arrays to
        - file_name: str - the name of the input csv file
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done).
                Defaults to `False`
    """
    if os.path.exists(out) and not force:
        print("File already exists, not doing any work")
        return

    df = pandas.read_csv(file_name)

    inputs = np.zeros((len(df), NUM_REACTIVITIES, NUM_BASES))
    outputs = np.zeros((len(df), NUM_REACTIVITIES))
    output_masks = np.ones((len(df), NUM_REACTIVITIES), dtype=np.bool_)
    errors = np.zeros((len(df), NUM_REACTIVITIES))

    for index in tqdm(range(len(df))):
        row = df.iloc[index]

        # get the sequence
        seq_len = len(row["sequence"])

        # map the base to its one-hot encoding
        inputs[index, :seq_len] = np.array(
            list(map(lambda letter: base_map[letter], row["sequence"]))
        )

        # get all the reactivities and reactivity errors
        reactivities = np.array(
            list(
                map(
                    lambda seq_idx: row["reactivity_" + str(seq_idx + 1).rjust(4, "0")],
                    range(seq_len),
                )
            )
        )
        reactivity_errors = np.array(
            list(
                map(
                    lambda seq_idx: row["reactivity_error_" + str(seq_idx + 1).rjust(4, "0")],
                    range(seq_len),
                )
            )
        )

        # replace reactivity error nans with 0s (assume no error)
        reactivity_errors = np.where(np.isnan(reactivity_errors), 0.0, reactivity_errors)

        # get where all the reactivities are nan
        nan_locats = np.isnan(reactivities)

        # where it is nan, store True, else false
        output_masks[index, :seq_len] = nan_locats

        # where it is not nan, store the reactivity and error, else 0
        outputs[index, :seq_len] = np.where(nan_locats == False, reactivities, 0.0)
        errors[index, :seq_len] = np.where(nan_locats == False, reactivity_errors, 0.0)

    # save the outputs
    np.savez_compressed(out, inputs=inputs, outputs=outputs, output_masks=output_masks, errors=errors)

In [9]:
preprocess_csv("train_data_2a3_preprocessed.npz", "train_data_2a3.csv")

File already exists, not doing any work


In [10]:
preprocess_csv("train_data_dms_preprocessed.npz", "train_data_dms.csv")

File already exists, not doing any work


### Load the desired dataset

In [11]:
desired_dataset = "2a3"  # either "2a3" or "dms"

In [12]:
# load the npz file
npz_file = np.load(f"train_data_{desired_dataset}_preprocessed.npz")

# stored inputs, outputs, and output_masks
# note: if visualizing, you may just want to only load outputs and bool_output_masks
# since histplot takes a lot of RAM.
inputs, outputs, bool_output_masks, errors = (
    npz_file["inputs"],
    npz_file["outputs"],
    npz_file["output_masks"],
    npz_file["errors"]
)

# close the npz file
npz_file.close()

In [13]:
# convert to usable weights:
# if it is meant to be masked, it should be worth 0, else it should be worth 1 - error
output_masks = np.where(bool_output_masks, 0., 1.)
output_masks -= errors
output_masks[output_masks < 0.] = 0.
output_masks.shape

(210992, 457)

In [14]:
outputs = np.clip(outputs, 0, 1)

### Visualize

This section of the notebook allows for visualizing the reactivities of the
current dataset.

In [15]:
visualize = False

In [16]:
if visualize:
    visualized_items = []
    for i in tqdm(range(len(outputs))):
        for x in range(NUM_REACTIVITIES):
            if not bool_output_masks[i, x]:
                visualized_items.append(outputs[i, x])
    visualized_items = np.array(visualized_items)
    print(f"took {len(visualized_items)}/{len(outputs)*NUM_REACTIVITIES} reactivities")
else:
    print("Not visualizing. Set `visualize` to `True` to visualize data")

Not visualizing. Set `visualize` to `True` to visualize data


In [17]:
if visualize:
    seaborn.histplot(visualized_items, binwidth=0.1)
else:
    print("Not visualizing. Set `visualize` to `True` to visualize data")

Not visualizing. Set `visualize` to `True` to visualize data


## Model

In [18]:
def make_baseline_model():
    inputs = layers.Input((NUM_REACTIVITIES, NUM_BASES))

    x = layers.Conv1D(4, 16, 1, padding="same", activation="relu")(inputs)
    x = layers.Conv1D(4, 16, 1, padding="same", activation="relu")(x)
    x = layers.Conv1D(4, 16, 2, padding="same", activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)

    x = layers.Dense(NUM_REACTIVITIES, activation='relu')(x)

    return keras.Model(inputs=inputs, outputs=x)

def load_model() -> keras.Model:
    return keras.models.load_model(f"{desired_dataset}_model")

In [19]:
model = make_baseline_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 457, 4)]          0         
                                                                 
 conv1d (Conv1D)             (None, 457, 4)            260       
                                                                 
 conv1d_1 (Conv1D)           (None, 457, 4)            260       
                                                                 
 conv1d_2 (Conv1D)           (None, 229, 4)            260       
                                                                 
 flatten (Flatten)           (None, 916)               0         
                                                                 
 dense (Dense)               (None, 1024)              939008    
                                                                 
 dense_1 (Dense)             (None, 457)               468425

2023-10-06 11:55:52.658506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-06 11:55:52.790010: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-06 11:55:52.790322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [20]:
# compile the model with an optimizer and MAE loss
model.compile(optimizer=optimizers.Adam(1e-4))

## Train

In [21]:
@tf.function
def train_batch(m: keras.Model, inps: tf.Tensor, outs: tf.Tensor, masks: tf.Tensor):
    """
    Get the loss on a batch and perform the corresponding weight updates.
    Used for training purposes
    """

    with tf.GradientTape() as tape:
        preds = tf.expand_dims(m(inps, training=True), axis=-1)
        outs = tf.expand_dims(outs, axis=-1)
        mae = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM)

        loss = 0.0
        for b in range(inps.shape[0]):
            loss += mae(outs[b], preds[b], sample_weight=masks[b])

        # turn it into mean
        loss /= inps.shape[0]
    
        # add the regularization losses
        regularization_loss = tf.add_n(m.losses)
        loss += regularization_loss
        
        # calculate gradients
        grads = tape.gradient(loss, m.trainable_variables)

    # apply grads
    m.optimizer.apply_gradients(zip(grads, m.trainable_variables))

    # return total loss, mae loss
    return loss, loss - regularization_loss


@tf.function
def noupdate_batch(m: keras.Model, inps: tf.Tensor, outs: tf.Tensor, masks: tf.Tensor):
    """
    Get the loss on a batch without performing any updates.
    Used for validation purposes
    """
    preds = tf.expand_dims(m(inps, training=False), axis=-1)
    outs = tf.expand_dims(outs, axis=-1)

    loss = 0.0
    mae = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM)
    for b in range(inps.shape[0]):
        loss += mae(preds[b], outs[b], sample_weight = masks[b])

    # turn it into mean
    loss /= inps.shape[0]

    # return mae loss
    return loss


def masked_train(
    m: keras.Model,
    x: np.ndarray,
    y: np.ndarray,
    masks: np.ndarray,
    batch_size: int = 32,
    epochs: int = 1,
    validation_split: float = 0.1,
):
    """
    Train the given model.

    Arguments:
        - m: keras.Model - the model to train.
        - x: np.ndarray - the numpy array of inputs.
        - y: np.ndarray - the numpy array of outputs.
        - masks: np.ndarray - the sample weights (1s and 0s).
        - batch_size: int - how large the batches should be. Defaults to `32`.
        - epochs: int - how many epochs to train for. Defaults to `1`.
        - validation_split: float - how large the validation subset should be, in the range (0, 1]. Defaults to `0.1`.
    
    Note - The choice of np.ndarray is purely arbitrary, and this function can be modified to use tf.Tensors
    
    Note - shuffle code is provided in numpy, but commented out because of memory limitations that less powerful computers
    may encounter.
    """
    # shuffle
    # shuffled_idxs = np.arange(x.shape[0])
    # np.random.shuffle(shuffled_idxs)
    # x = x[shuffled_idxs]
    # y = y[shuffled_idxs]
    # masks = masks[shuffled_idxs]

    # generate validation
    validation_size = int(x.shape[0] * validation_split)
    x_val = x[:validation_size]
    y_val = y[:validation_size]
    masks_val = masks[:validation_size]
    x = x[validation_size:]
    y = y[validation_size:]
    masks = masks[validation_size:]

    # calculate number of batches to do
    num_batches = x.shape[0] // batch_size
    if x.shape[0] % batch_size != 0:
        num_batches += 1

    num_validation_batches = x_val.shape[0] // batch_size
    if x_val.shape[0] % batch_size != 0:
        num_validation_batches += 1

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}")
        epoch_loss = 0.0
        epoch_mae = 0.0

        for batch in range(num_batches):
            num_items = min(batch_size, x.shape[0] - batch * batch_size)

            inps = tf.constant(x[batch * batch_size : batch * batch_size + num_items])
            outs = tf.constant(y[batch * batch_size : batch * batch_size + num_items])
            masks_ = tf.constant(
                masks[batch * batch_size : batch * batch_size + num_items]
            )

            loss, mae_loss = train_batch(m, inps, outs, masks_)

            epoch_loss += loss
            epoch_mae += mae_loss

            # log
            print(
                f"Batch {batch+1}/{num_batches}\t- loss: {loss.numpy():.5f}\t- mae loss: {mae_loss.numpy():.5f}",
                end="\r",
            )
        epoch_loss /= num_batches
        epoch_mae /= num_batches

        # do validation
        val_mae = 0.0
        for batch in range(num_validation_batches):
            num_items = min(batch_size, x_val.shape[0] - batch * batch_size)

            inps = tf.constant(
                x_val[batch * batch_size : batch * batch_size + num_items]
            )
            outs = tf.constant(
                y_val[batch * batch_size : batch * batch_size + num_items]
            )
            masks_ = tf.constant(
                masks_val[batch * batch_size : batch * batch_size + num_items]
            )

            mae_loss = noupdate_batch(m, inps, outs, masks_)

            val_mae += mae_loss
        val_mae /= num_validation_batches

        # shuffle
        # shuffled_idxs = np.arange(x.shape[0])
        # np.random.shuffle(shuffled_idxs)
        # x = x[shuffled_idxs]
        # y = y[shuffled_idxs]
        # masks = masks[shuffled_idxs]

        print()
        print(
            f"Epoch loss: {epoch_loss:.5f}\tEpoch MAE: {epoch_mae:.5f}\tVal MAE: {val_mae:.5f}"
        )

In [22]:
masked_train(model, inputs, outputs, output_masks, epochs=20, batch_size=512)

Epoch 1


2023-10-06 11:56:10.599639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-10-06 11:56:11.195355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Batch 371/371	- loss: 31.08087	- mae loss: 31.06038
Epoch loss: 21.88480	Epoch MAE: 21.85213	Val MAE: 24.92956
Epoch 2
Batch 371/371	- loss: 26.99781	- mae loss: 26.98098
Epoch loss: 21.50853	Epoch MAE: 21.48919	Val MAE: 24.37587
Epoch 3
Batch 371/371	- loss: 21.69537	- mae loss: 21.66579
Epoch loss: 21.06756	Epoch MAE: 21.04674	Val MAE: 24.07790
Epoch 4
Batch 371/371	- loss: 15.15701	- mae loss: 15.09221
Epoch loss: 20.59314	Epoch MAE: 20.56321	Val MAE: 23.66617
Epoch 5
Batch 371/371	- loss: 12.04827	- mae loss: 11.95378
Epoch loss: 20.20325	Epoch MAE: 20.16215	Val MAE: 23.24121
Epoch 6
Batch 371/371	- loss: 10.94260	- mae loss: 10.83698
Epoch loss: 19.92563	Epoch MAE: 19.87944	Val MAE: 23.07961
Epoch 7
Batch 371/371	- loss: 10.43401	- mae loss: 10.32466
Epoch loss: 19.69594	Epoch MAE: 19.64114	Val MAE: 22.88339
Epoch 8
Batch 371/371	- loss: 10.12862	- mae loss: 10.01545
Epoch loss: 19.54281	Epoch MAE: 19.48711	Val MAE: 22.73209
Epoch 9
Batch 371/371	- loss: 10.10537	- mae loss: 10.00

## Save

This section saves the current model

In [23]:
model.save(f"{desired_dataset}_model")

INFO:tensorflow:Assets written to: 2a3_model/assets


INFO:tensorflow:Assets written to: 2a3_model/assets


## Process Outputs

This section of the noteboook creates a zipped csv submission file that can
be submitted on Kaggle.

In [24]:
make_submissions = True

In [25]:
valid = False

if os.path.exists("2a3_model") and os.path.exists("dms_model") and os.path.exists("test_sequences.csv") and make_submissions:
    valid = True
    model_2a3 = keras.models.load_model("2a3_model")
    model_dms = keras.models.load_model("dms_model")
else:
    print("Not going to create submissions.")

In [26]:
@tf.function
def call_model(model_2a3, model_dms, inputs):
    return model_2a3(inputs, training=False), model_dms(inputs, training=False)


def pipeline(
    model_2a3: keras.Model,
    model_dms: keras.Model,
    input_csv: str,
    out: str,
    batch_size: int,
):
    """
    Process test data and write submissions to a csv file

    Arguments:
        - model_2a3: keras.Model - the model trained on the 2a3 distribution
        - model_dms: keras.Model - the model trained on the dms distribution
        - input_csv: str - the name of the file that contains the test data 
        - out: str - the name of the file to write predictions to
        - batch_size: int - how many predictions to make at a time
    """
    count = 0

    # count how many lines we have in total
    with open(input_csv) as file:
        line = file.readline()  # ignore the header
        # take the first line since we increment count in the loop
        line = file.readline()
        while line != "":
            count += 1
            line = file.readline()

    # use that knowledge for a progress bar
    with open(input_csv, "r") as file, open(out, "w") as outfile:
        # write the header
        outfile.write("id,reactivity_DMS_MaP,reactivity_2A3_MaP\n")

        # get what index the things we need are
        header = file.readline()
        split_header = header.split(",")
        min_idx = split_header.index("id_min")
        max_idx = split_header.index("id_max")
        sequence_idx = split_header.index("sequence")

        # only take the approved filtered lines
        num_batches = count // batch_size
        if count % batch_size != 0:
            num_batches += 1
        for batch in tqdm(range(num_batches)):
            num_items = min(batch_size, count - batch * batch_size)

            # initialize variables
            inputs = np.zeros((num_items, NUM_REACTIVITIES, NUM_BASES))
            min_seq_idxs = []
            sequence_lengths = []

            # collect the inputs
            for i in range(num_items):
                line = file.readline()
                temp = line.split(",")
                sequence = temp[sequence_idx]
                max_seq_idx = int(temp[max_idx])
                min_seq_idx = int(temp[min_idx])

                # verify that everything is correct
                assert len(sequence) + min_seq_idx - 1 == max_seq_idx

                # store the data
                inputs[i, : len(sequence)] = np.array(
                    list(map(lambda letter: base_map[letter], sequence))
                )
                min_seq_idxs.append(min_seq_idx)
                sequence_lengths.append(len(sequence))

            # run inputs through the associated model
            probs_2a3, probs_dms = call_model(model_2a3, model_dms, inputs)
            probs_dms = probs_dms.numpy()
            probs_2a3 = probs_2a3.numpy()

            # write predictions
            for i in range(num_items):
                for seq_idx in range(
                    min_seq_idxs[i], min_seq_idxs[i] + sequence_lengths[i]
                ):
                    outfile.write(
                        f"{seq_idx},{probs_dms[i, seq_idx - min_seq_idxs[i]]:.3f},{probs_2a3[i, seq_idx - min_seq_idxs[i]]:.3f}\n"
                    )

In [27]:
if valid:
    pipeline(
        model_2a3, model_dms, "test_sequences.csv", "submission.csv", batch_size=256
    )
else:
    print("Not going to create submissions.")

100%|██████████| 5250/5250 [04:42<00:00, 18.56it/s]


In [29]:
if valid:
    # zip our submission into an easily-uploadable zip file
    print("zipping submissions. This may take a while...")
    os.system("zip submission.csv.zip submission.csv")
    print("Done zipping submissions!")
else:
    print("Not going to zip submissions.")

zipping submissions. This may take a while...
  adding: submission.csv